In [1]:
import numpy as np
from parser import parse_model_from_txt

# Simplex

In [2]:
class Simplex:
    def __init__(self, c_from_parser, A_from_parser, b, signs, was_min=False, interpretation_info=None):
        # Estes são os coeficientes e matriz para variáveis já transformadas pelo parser (não-negativas)
        self.c_parser_vars = np.array(c_from_parser, dtype=float)
        self.A_parser_vars = np.array(A_from_parser, dtype=float) # Dimensões: m x n_parser_vars
        self.b_orig = np.array(b, dtype=float)                    # Vetor m x 1
        self.signs = signs
        self.was_min = was_min
        self.interpretation_info = interpretation_info
        self.m, self.n_parser_vars = self.A_parser_vars.shape

        self.solution = None
        self.status = None

        # Atributos para o Simplex Tabular (inicializados por _build_phase_one)
        self.tableau = None
        self.basic_vars = None # Para tabular: lista de índices de colunas no tableau
        self.num_vars = None   # Para tabular: número total de colunas de variáveis no tableau
        self.c_phase1 = None   # Para tabular: custos da fase 1
        self.artificial_indices = [] # Para tabular: índices das colunas artificiais no tableau


    def _build_phase_one(self):
        # Esta é a sua função _build_phase_one tabular.
        # Ela inicializa self.tableau, self.basic_vars, self.num_vars, 
        # self.c_phase1, self.artificial_indices.
        # Ela usa self.A_parser_vars e self.c_parser_vars como base.
        
        m, n_orig_transformed = self.A_parser_vars.shape
        num_slack_total = 0
        num_artificial_total = 0
        # ... (lógica de contagem de vars de folga/artificiais como na versão corrigida) ...
        for sign in self.signs:
            if sign == '<=':
                num_slack_total += 1
            elif sign == '>=':
                num_slack_total += 1
                num_artificial_total += 1
            elif sign == '=':
                num_artificial_total += 1
        
        self.num_vars = n_orig_transformed + num_slack_total + num_artificial_total # Num total de cols no tableau
        self.tableau = np.zeros((m, self.num_vars + 1))
        self.artificial_indices = [] # Índices no tableau
        current_basic_vars_tabular = [-1] * m

        slack_col_ptr = n_orig_transformed
        artificial_col_ptr = n_orig_transformed + num_slack_total

        for i in range(m):
            self.tableau[i, :n_orig_transformed] = self.A_parser_vars[i, :]
            self.tableau[i, -1] = self.b_orig[i]
            sign = self.signs[i]
            if sign == '<=':
                self.tableau[i, slack_col_ptr] = 1.0
                current_basic_vars_tabular[i] = slack_col_ptr
                slack_col_ptr += 1
            elif sign == '>=':
                self.tableau[i, slack_col_ptr] = -1.0 # Excesso
                slack_col_ptr += 1
                self.tableau[i, artificial_col_ptr] = 1.0 # Artificial
                self.artificial_indices.append(artificial_col_ptr)
                current_basic_vars_tabular[i] = artificial_col_ptr
                artificial_col_ptr += 1
            elif sign == '=':
                self.tableau[i, artificial_col_ptr] = 1.0 # Artificial
                self.artificial_indices.append(artificial_col_ptr)
                current_basic_vars_tabular[i] = artificial_col_ptr
                artificial_col_ptr += 1
        
        self.basic_vars = current_basic_vars_tabular
        self.c_phase1 = np.zeros(self.num_vars)
        for art_idx_tableau in self.artificial_indices:
            self.c_phase1[art_idx_tableau] = -1.0 # Maximizando -Soma(Artificiais)

        if any(bv == -1 for bv in self.basic_vars):
             raise ValueError(f"Tabular _build_phase_one: Nem todas as variáveis básicas foram detectadas: {self.basic_vars}")


    def _simplex(self, c_tableau, phase): # Método tabular existente
        # ... (seu código _simplex tabular corrigido, que opera em self.tableau, 
        #      self.basic_vars, self.num_vars e usa c_tableau como vetor de custos) ...
        # Este método modifica self.tableau e self.basic_vars in-loco.
        # Certifique-se que ele retorna um dicionário {'status': ..., 'solution': ..., 'value': ...}
        # A 'solution' retornada por este _simplex será para todas as self.num_vars do tableau.
        m, n_plus_1 = self.tableau.shape 
        current_tableau_num_vars = n_plus_1 - 1

        # Garantir que c_tableau tenha o tamanho correto
        if len(c_tableau) != current_tableau_num_vars:
            # Ajustar c_tableau se necessário, ou levantar erro
            # Esta situação pode ocorrer se c_phase1 ou self.c não estiverem alinhados com self.num_vars
            # No setup tabular, c_phase1 e self.c (para phase 2) devem ter self.num_vars elementos
            temp_c = np.zeros(current_tableau_num_vars)
            temp_c[:len(c_tableau)] = c_tableau
            c_tableau = temp_c
            # raise ValueError(f"Comprimento do vetor de custo ({len(c_tableau)}) não corresponde ao número de variáveis no tableau ({current_tableau_num_vars})")


        cb = c_tableau[self.basic_vars] 
        
        iteration = 0
        max_iterations = self.m * self.num_vars * 2 # Heurística para limite de iterações

        while iteration < max_iterations:
            zj = cb @ self.tableau[:, :-1]
            cj_zj = c_tableau - zj

            if np.all(cj_zj <= 1e-8):
                solution_tableau = np.zeros(current_tableau_num_vars)
                for i, bi_col_idx in enumerate(self.basic_vars):
                    if bi_col_idx < current_tableau_num_vars : 
                        solution_tableau[bi_col_idx] = self.tableau[i, -1]
                
                current_obj_value = c_tableau @ solution_tableau
                return {'status': 'optimal', 'solution': solution_tableau, 'value': current_obj_value}

            entering_col = np.argmax(cj_zj)
            
            if np.all(self.tableau[:, entering_col] <= 1e-8):
                return {'status': 'unbounded', 'solution': None, 'value': None}

            ratios = np.full(self.m, np.inf)
            for i in range(self.m):
                if self.tableau[i, entering_col] > 1e-8:
                    if self.tableau[i, -1] >= -1e-8: # rhs não-negativo
                         ratios[i] = self.tableau[i, -1] / self.tableau[i, entering_col]
            
            if np.all(ratios == np.inf):
                 return {'status': 'unbounded', 'solution': None, 'value': None} # Segurança

            leaving_row = np.argmin(ratios)
            
            pivot_element = self.tableau[leaving_row, entering_col]
            if abs(pivot_element) < 1e-9: # Pivô muito pequeno, instabilidade
                return {'status': 'error_pivot_too_small', 'solution': None, 'value': None}

            self.tableau[leaving_row, :] /= pivot_element
            
            for i in range(self.m):
                if i != leaving_row:
                    factor = self.tableau[i, entering_col]
                    self.tableau[i, :] -= factor * self.tableau[leaving_row, :]
            
            self.basic_vars[leaving_row] = entering_col
            cb = c_tableau[self.basic_vars]
            iteration += 1
        
        return {'status': 'max_iterations_reached', 'solution': None, 'value': None}


    def solve(self, method='tabular'):
        if method == 'tabular':
            self._build_phase_one()
            result_phase1 = self._simplex(self.c_phase1, phase=1)

            if result_phase1 is None or \
                result_phase1.get('status') != 'optimal' or \
                abs(result_phase1.get('value', float('inf'))) > 1e-6: # Epsilon para FO da Fase 1
                self.status = result_phase1.get('status', 'infeasible') if result_phase1 else 'infeasible'
                return {'status': self.status, 'solution': None, 'value': None}

            # Preparar para Fase 2 (Tabular)
            num_vars_before_deletion = self.num_vars
            vars_to_keep_tableau = [idx for idx in range(num_vars_before_deletion) if idx not in self.artificial_indices]
            
            # Mapear basic_vars para novos índices
            new_basic_vars_tabular = []
            map_old_to_new_idx_tableau = {old_idx: new_idx for new_idx, old_idx in enumerate(vars_to_keep_tableau)}

            for bv_old_idx in self.basic_vars:
                if bv_old_idx in self.artificial_indices: 
                    # Idealmente, artificiais não devem estar na base se FO da Fase 1 é 0.
                    # Se estiver, e com valor 0, precisa de tratamento especial (pivotar para fora).
                    # Se estiver com valor > 0, a checagem da FO da Fase 1 deveria ter pego.
                    # Para esta implementação, assumimos que não estão na base ou são 0 e podem ser ignoradas.
                    # Uma implementação mais robusta lidaria com isso explicitamente.
                    continue # Não incluir artificiais na base da Fase 2
                
                # Checar se a var básica ainda existe após deleção conceitual das artificiais
                if bv_old_idx in map_old_to_new_idx_tableau:
                    new_basic_vars_tabular.append(map_old_to_new_idx_tableau[bv_old_idx])
                # else: uma variável básica foi removida (não deveria acontecer se não for artificial)

            self.tableau = self.tableau[:, vars_to_keep_tableau + [num_vars_before_deletion]] # Mantém colunas e RHS
            self.num_vars = len(vars_to_keep_tableau)
            
            # Reconstruir basic_vars se o tamanho mudou devido a artificiais básicas removidas
            # Esta parte é complexa. Se len(new_basic_vars_tabular) != self.m, a base foi comprometida.
            # Por agora, se a contagem for correta, usamos. Senão, pode haver erro.
            if len(new_basic_vars_tabular) == self.m :
                    self.basic_vars = new_basic_vars_tabular
            else: # A base ficou inválida. Isso indica um problema.
               raise Exception("Base tabular inválida após remoção de vars artificiais para Fase 2")


            self.c = np.zeros(self.num_vars)
            len_parser_vars = len(self.c_parser_vars)
            # As vars do parser são as primeiras entre as 'vars_to_keep_tableau'
            # Isso assume que c_parser_vars corresponde às primeiras colunas das vars não artificiais
            
            # Contar quantas vars do parser realmente restaram
            num_parser_vars_remaining = 0
            for i in range(len_parser_vars):
                # O parser_var original corresponde a qual coluna em vars_to_keep_tableau?
                # Isso é complexo porque c_parser_vars é para as vars transformadas pelo parser,
                # e vars_to_keep_tableau é para as colunas do tableau (parser + folga/excesso).
                # O mais simples é que c_parser_vars são os custos das primeiras len_parser_vars
                # colunas de vars_to_keep_tableau (se a ordem for mantida).
                if i < self.num_vars : # Garante que não exceda o número de vars na Fase 2
                        self.c[i] = self.c_parser_vars[i]
            
            result_phase2 = self._simplex(self.c, phase=2)

        else:
            raise ValueError("Método Simplex desconhecido. Escolha 'tabular' ou 'revised'.")

        # Pós-processamento comum
        if result_phase2 and result_phase2.get('status') == 'optimal' and self.was_min:
            result_phase2['value'] = -result_phase2['value']
        
        if result_phase2 and result_phase2.get('solution') is not None and self.interpretation_info:
            solution_values_from_solver = result_phase2['solution']
            final_solution_orig_vars = {}
            # sorted_original_vars vem de interpretation_info
            for var_orig_name in self.interpretation_info['sorted_original_vars']:
                map_info_parser = self.interpretation_info['simplex_vars_map'][var_orig_name]
                # cols_parser são os índices das vars transformadas pelo parser
                # Estes são os primeiros N índices em solution_values_from_solver,
                # antes das vars de folga/excesso (tabular) ou das outras vars conceituais (MSR)
                cols_in_parser_level_vars = map_info_parser['cols_parser']

                val_to_assign = 0
                if map_info_parser['type'] == 'free':
                    idx_p = cols_in_parser_level_vars[0]
                    idx_n = cols_in_parser_level_vars[1]
                    val_p = solution_values_from_solver[idx_p] if idx_p < len(solution_values_from_solver) else 0
                    val_n = solution_values_from_solver[idx_n] if idx_n < len(solution_values_from_solver) else 0
                    val_to_assign = val_p - val_n
                elif map_info_parser['type'] == 'negative':
                    idx_prime = cols_in_parser_level_vars[0]
                    val_prime = solution_values_from_solver[idx_prime] if idx_prime < len(solution_values_from_solver) else 0
                    val_to_assign = -val_prime
                else: # non_negative
                    idx_orig = cols_in_parser_level_vars[0]
                    val_non_neg = solution_values_from_solver[idx_orig] if idx_orig < len(solution_values_from_solver) else 0
                    val_to_assign = val_non_neg
                final_solution_orig_vars[var_orig_name] = val_to_assign
            
            ordered_solution_list = [final_solution_orig_vars[v_name] for v_name in self.interpretation_info['sorted_original_vars']]
            result_phase2['solution'] = ordered_solution_list
        
        return result_phase2

# Converter Modelo no TXT

In [20]:
model = parse_model_from_txt(r'./modelos/teste.txt')
print(model)


{'c': [1.0, 2.0, -1.0], 'A': [[2.0, 1.0, 1.0], [4.0, 2.0, 3.0], [2.0, 5.0, 5.0]], 'b': [14.0, 28.0, 30.0], 'signs': ['<=', '<=', '<='], 'was_min': False, 'interpretation_info': {'sorted_original_vars': ['x1', 'x2', 'x3'], 'simplex_vars_map': {'x1': {'type': 'non_negative', 'cols_parser': [0], 'mult': 1, 'simplex_names': ['x1']}, 'x2': {'type': 'non_negative', 'cols_parser': [1], 'mult': 1, 'simplex_names': ['x2']}, 'x3': {'type': 'non_negative', 'cols_parser': [2], 'mult': 1, 'simplex_names': ['x3']}}, 'simplex_var_column_names': ['x1', 'x2', 'x3']}}


In [21]:
simplex = Simplex(model['c'], model['A'], 
                  model['b'], model['signs'], was_min=model['was_min'], 
                  interpretation_info=model['interpretation_info'])

result = simplex.solve()
print(result)

{'status': 'optimal', 'solution': [5.0, 4.0, 0.0], 'value': 13.0}
